In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import defaultdict
import urllib.request
import csv
import io

In [14]:
url = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/groceries.csv"

try:
    response = urllib.request.urlopen(url)
    dataset = response.read().decode('utf-8')
    print("Dataset loaded successfully!")
except Exception as e:
    print(f"Error loading dataset: {e}")

Dataset loaded successfully!


In [16]:
# Parse the CSV data
transactions = []
for line in csv.reader(io.StringIO(dataset)):
    # Filter out empty strings
    transaction = [item for item in line if item.strip()]
    if transaction:  # Only add non-empty transactions
        transactions.append(transaction)

print(f"Total number of transactions: {len(transactions)}")
print("\nSample transactions:")
for i in range(min(5, len(transactions))):
    print(transactions[i])

Total number of transactions: 9835

Sample transactions:
['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups']
['tropical fruit', 'yogurt', 'coffee']
['whole milk']
['pip fruit', 'yogurt', 'cream cheese ', 'meat spreads']
['other vegetables', 'whole milk', 'condensed milk', 'long life bakery product']


In [18]:
class ManualApriori:
    def __init__(self, min_support=0.01, min_confidence=0.6):
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.frequent_itemsets = {}
        self.transaction_count = 0
        self.item_counts = {}

    def _init_pass(self, transactions):
        """Generate C1: candidate 1-itemsets"""
        self.transaction_count = len(transactions)
        C1 = {}
        for transaction in transactions:
            for item in set(transaction):
                if item in C1:
                    C1[item] += 1
                else:
                    C1[item] = 1
        return C1

    def _is_frequent(self, itemset_count):
        """Check if the itemset meets minimum support"""
        return itemset_count / self.transaction_count >= self.min_support

    def _candidate_gen(self, Fk_1):
        """Generate candidate k-itemsets from frequent (k-1)-itemsets"""
        Ck = {}
        Fk_1_list = list(Fk_1.keys())
        k = len(Fk_1_list[0]) + 1 if Fk_1_list else 1

        for i in range(len(Fk_1_list)):
            for j in range(i+1, len(Fk_1_list)):
                items1 = sorted(Fk_1_list[i])
                items2 = sorted(Fk_1_list[j])

                if k > 2 and items1[:-1] != items2[:-1]:
                    continue

                new_candidate = tuple(sorted(set(items1) | set(items2)))
                if len(new_candidate) == k:
                    Ck[new_candidate] = 0

        Ck_pruned = Ck.copy()
        for candidate in Ck:
            for i in range(k):
                subset = tuple(sorted(list(candidate)[:i] + list(candidate)[i+1:]))
                if subset not in Fk_1:
                    if candidate in Ck_pruned:
                        del Ck_pruned[candidate]
                    break

        return Ck_pruned

    def find_frequent_itemsets(self, transactions):
        """Find all frequent itemsets using the Apriori algorithm"""
        start_time = time.time()


        C1 = self._init_pass(transactions)

        self.item_counts = {(item,): count for item, count in C1.items()}


        F1 = {(item,): count for item, count in C1.items() if self._is_frequent(count)}
        self.frequent_itemsets[1] = F1

        print(f"Found {len(F1)} frequent 1-itemsets")

        k = 2
        while self.frequent_itemsets[k-1]:

            Ck = self._candidate_gen(self.frequent_itemsets[k-1])

            print(f"Generated {len(Ck)} candidate {k}-itemsets")

            for transaction in transactions:
                transaction_set = set(transaction)
                for candidate in Ck:
                    if set(candidate).issubset(transaction_set):
                        Ck[candidate] += 1

            Fk = {candidate: count for candidate, count in Ck.items()
                  if self._is_frequent(count)}

            if Fk:
                self.frequent_itemsets[k] = Fk
                print(f"Found {len(Fk)} frequent {k}-itemsets")
                k += 1
            else:
                break

            print(f"Iteration {k-1} completed in {time.time() - start_time:.2f} seconds")

        total_frequent_itemsets = sum(len(itemsets) for itemsets in self.frequent_itemsets.values())
        print(f"Total frequent itemsets found: {total_frequent_itemsets}")

        return self.frequent_itemsets

    def _get_all_subsets(self, itemset):
        """Get all non-empty proper subsets of an itemset"""
        n = len(itemset)
        subsets = []
        for i in range(1, 2**n - 1):
            subset = tuple(sorted(itemset[j] for j in range(n) if (i & (1 << j))))
            subsets.append(subset)
        return subsets

    def generate_rules(self):
        """Generate association rules from frequent itemsets"""
        print("\nGenerating association rules...")
        rules = []
        rule_count = 0

        for k in range(2, len(self.frequent_itemsets) + 1):
            for itemset, itemset_count in self.frequent_itemsets[k].items():
                itemset_support = itemset_count / self.transaction_count

                subsets = self._get_all_subsets(itemset)

                for antecedent in subsets:
                    consequent = tuple(sorted(set(itemset) - set(antecedent)))

                    if not consequent:
                        continue

                    ant_support_count = 0
                    for size, itemsets in self.frequent_itemsets.items():
                        if size == len(antecedent) and antecedent in itemsets:
                            ant_support_count = itemsets[antecedent]
                            break

                    ant_support = ant_support_count / self.transaction_count

                    confidence = itemset_support / ant_support

                    if confidence >= self.min_confidence:
                        cons_support_count = 0
                        for size, itemsets in self.frequent_itemsets.items():
                            if size == len(consequent) and consequent in itemsets:
                                cons_support_count = itemsets[consequent]
                                break

                        cons_support = cons_support_count / self.transaction_count
                        lift = confidence / cons_support

                        rules.append({
                            'antecedent': antecedent,
                            'consequent': consequent,
                            'support': itemset_support,
                            'confidence': confidence,
                            'lift': lift
                        })
                        rule_count += 1

        print(f"Total rules generated: {len(rules)}")
        return rules


In [19]:
print("\nRunning manual Apriori implementation...")
start_time = time.time()
apriori_manual = ManualApriori(min_support=0.01, min_confidence=0.50)
frequent_itemsets = apriori_manual.find_frequent_itemsets(transactions)
total_time = time.time() - start_time
print(f"Frequent itemset generation completed in {total_time:.2f} seconds")


Running manual Apriori implementation...
Found 88 frequent 1-itemsets
Generated 3828 candidate 2-itemsets
Found 213 frequent 2-itemsets
Iteration 2 completed in 7.48 seconds
Generated 576 candidate 3-itemsets
Found 32 frequent 3-itemsets
Iteration 3 completed in 8.70 seconds
Generated 6 candidate 4-itemsets
Total frequent itemsets found: 333
Frequent itemset generation completed in 8.72 seconds


In [20]:
start_time = time.time()
rules = apriori_manual.generate_rules()
rule_gen_time = time.time() - start_time
print(f"Rule generation completed in {rule_gen_time:.2f} seconds")


Generating association rules...
Total rules generated: 15
Rule generation completed in 0.00 seconds


In [21]:
print("\nSample frequent itemsets (support count):")
for k in range(1, min(4, len(frequent_itemsets) + 1)):
    print(f"\n{k}-itemsets (showing up to 5):")
    for idx, (items, count) in enumerate(list(frequent_itemsets[k].items())[:5]):
        print(f"  {items}: {count} (support: {count/apriori_manual.transaction_count:.4f})")
    if len(frequent_itemsets[k]) > 5:
        print(f"  ... and {len(frequent_itemsets[k]) - 5} more")


Sample frequent itemsets (support count):

1-itemsets (showing up to 5):
  ('semi-finished bread',): 174 (support: 0.0177)
  ('margarine',): 576 (support: 0.0586)
  ('citrus fruit',): 814 (support: 0.0828)
  ('coffee',): 571 (support: 0.0581)
  ('yogurt',): 1372 (support: 0.1395)
  ... and 83 more

2-itemsets (showing up to 5):
  ('margarine', 'yogurt'): 140 (support: 0.0142)
  ('margarine', 'whole milk'): 238 (support: 0.0242)
  ('margarine', 'other vegetables'): 194 (support: 0.0197)
  ('margarine', 'rolls/buns'): 145 (support: 0.0147)
  ('bottled water', 'margarine'): 101 (support: 0.0103)
  ... and 208 more

3-itemsets (showing up to 5):
  ('citrus fruit', 'whole milk', 'yogurt'): 101 (support: 0.0103)
  ('citrus fruit', 'other vegetables', 'whole milk'): 128 (support: 0.0130)
  ('citrus fruit', 'other vegetables', 'root vegetables'): 102 (support: 0.0104)
  ('tropical fruit', 'whole milk', 'yogurt'): 149 (support: 0.0151)
  ('other vegetables', 'tropical fruit', 'yogurt'): 121 (s

In [23]:
print("\nTop 5 association rules by confidence:")
sorted_rules = sorted(rules, key=lambda x: x['confidence'], reverse=True)
for idx, rule in enumerate(sorted_rules[:10]):
    antecedent_str = ', '.join([f'"{item}"' for item in rule['antecedent']])
    consequent_str = ', '.join([f'"{item}"' for item in rule['consequent']])
    print(f"{idx+1}. {{{antecedent_str}}} => {{{consequent_str}}}")
    print(f"   Support: {rule['support']:.4f}, Confidence: {rule['confidence']:.4f}, Lift: {rule['lift']:.4f}")




Top 5 association rules by confidence:
1. {"citrus fruit", "root vegetables"} => {"other vegetables"}
   Support: 0.0104, Confidence: 0.5862, Lift: 3.0296
2. {"root vegetables", "tropical fruit"} => {"other vegetables"}
   Support: 0.0123, Confidence: 0.5845, Lift: 3.0210
3. {"curd", "yogurt"} => {"whole milk"}
   Support: 0.0101, Confidence: 0.5824, Lift: 2.2791
4. {"butter", "other vegetables"} => {"whole milk"}
   Support: 0.0115, Confidence: 0.5736, Lift: 2.2449
5. {"root vegetables", "tropical fruit"} => {"whole milk"}
   Support: 0.0120, Confidence: 0.5700, Lift: 2.2310
6. {"root vegetables", "yogurt"} => {"whole milk"}
   Support: 0.0145, Confidence: 0.5630, Lift: 2.2034
7. {"domestic eggs", "other vegetables"} => {"whole milk"}
   Support: 0.0123, Confidence: 0.5525, Lift: 2.1623
8. {"whipped/sour cream", "yogurt"} => {"whole milk"}
   Support: 0.0109, Confidence: 0.5245, Lift: 2.0527
9. {"rolls/buns", "root vegetables"} => {"whole milk"}
   Support: 0.0127, Confidence: 0.5230